In [21]:
import nltk
from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
import numpy as np
import re
import json
import random
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow import keras
from keras.models import load_model
import tkinter
from tkinter import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/test_drive_25_09_1976/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
lemmatizer = WordNetLemmatizer()
# Here read the json file
intents = json.loads(open('AccidentLevel.json').read())
model = load_model('modelBert.h5', custom_objects={'KerasLayer':hub.KerasLayer})

In [3]:
# Function to load AccidentLevel.json file
def loadjson(intents):
    dit = {}
    tagPattern = {}
    tagResponse = {}
    for content in intents['intents']:
        tag = content['tag']
        patrns = content['patterns']
        if (patrns == 'I' or patrns == 'II' or patrns == 'III' or patrns == 'IV' or patrns == 'V'):
            dit[patrns] = content['responses']
        else:
            tagPattern[tag] = content['patterns']
            tagResponse[tag] = content['responses']
    return dit, tagPattern, tagResponse

In [2]:
# Function for Data pre-processing or data cleaning
def clean_sentences(sentences):
    sent = (lambda sentences: re.sub(r'[^A-Za-z]+', ' ', sentences))(sentences)
    sent = (lambda sent: sent.lower())(sent)
    sent = (lambda sent: sent.strip())(sent)
    sent = (lambda sent: re.sub(r'[^\w\s]', '', sent))(sent)
    sent = (lambda sent: ' '.join([w for w in sent.split() if w not in stopwords]))(sent)
    return sent

In [ ]:
# Function to convert Accident level which is in integer to Roman numbers.
def convertIntToCategories(x):
    level = (lambda x: ('I' if x == 0 else ('II' if x == 1 else ('III' if x == 2 else ('IV' if x == 3 else 'V')))))(x)
    return level

In [ ]:
# This function returns Roman numbered accident level.
def getAccidentLevel(prediction):
    return convertIntToCategories(np.argmax(prediction))

In [ ]:
# This function anaylses the message. If the message is Intro message or Exit message then it would return bypass value 
# as True else False. This is done by looking the message in json file. If the message falls in Intro or Exit message patterns 
# then True is returned along with tag name.
def bypassPrediction(msg):
    bypass = False
    tag = ""
    for tagPatrn in tagPattern:
        patternList = tagPattern.get(tagPatrn)
        
        if any(patrn == msg.lower() for patrn in patternList):
            bypass = True
            tag = tagPatrn
            break
        else:
            bypass = False
            tag = ""
    return bypass, tag

In [ ]:
# This function returns user friendly message which is stored in json file for given accident level.
def getResponse(lvl, intents_json):
    return intents_json.get(lvl)

In [ ]:
# If the message is not among Intro or Exit messages then predict_class method is called. Inside this method clean_sentences
# method is invoked which would perform data cleansing on message. Then model.predict method is called which returns an 
# accident level in integers. The method getAccidentLevel converts accident level from integer to Roman number.
def predict_class(sentence, model):
    dataList = []
    data = clean_sentences(sentence)
    dataList.append(data)
    pred = model.predict(dataList)[0]
    return getAccidentLevel(pred)

In [ ]:
# This function would analyse the message. If the message is Intro message or Exit message then it would directly pickup 
# the response from AccidentLevel.json file & bypass the model.predict method call. If the message is other than Intro 
# message or Exit message then model.predict method call is invoked. The message anaylzing part is done in bypassPrediction 
# method. If bypass value is False then predict_class method is invoked to do model prediction else the response is pulled 
# from json file.
def chatbot_response(msg):
    bypass, tag = bypassPrediction(msg)
    if (bypass == True):
        res = tagResponse.get(tag)[0]
    else:
        lvl = predict_class(msg, model)
        res = getResponse(lvl, dits)
    return res

In [ ]:
# This function is invoked when a user hits send button. This function would invoke chatbot_response function.
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

In [25]:
# Routines to create UI using Trinker library
base = Tk()
base.title("ChatBot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff', command= send)

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")

#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()